In [23]:
import numpy as np
import os
import glob
import random
import json
import matplotlib.pyplot as plt
from obspy import read
from datetime import datetime, timedelta
from tqdm import tqdm

In [173]:
def getWavFile(infile, root_path):
    with open(infile) as f:
        lines = f.readlines()
        
    adict = {}
    adict['network'] = infile[-7:-4]
    adict['RecordLength'] = float(lines[3][-9:])
    adict['sampling_rate'] = int(lines[4][-5:])
    adict['datatype'] = 'Acceleration'
    
    z, n, e = [], [], []
    for i in lines[11:]:
        z.append(float(i[14:20]))
        n.append(float(i[24:30]))
        e.append(float(i[34:40]))
        
    z, n, e = np.array(z), np.array(n), np.array(e)
    
    # write to txt
    filepath = os.path.join(root_path, 'test')
    print(filepath)
    with open(filepath+'.txt', 'w') as file:
        file.write('\tZ\tN\tE\n')
        for i in range(z.shape[0]):
            file.write('\t{}\t{}\t{}\n'.format(z[i], n[i], e[i]))
    return adict

In [180]:
def unpackP(infile, root_path):
    with open(infile) as f:
        lines = f.readlines()
    
    # ================================ header ================================= #
    tmp = lines[0]
    year = int(tmp[1:5])
    month = int(tmp[5:7])
    day = int(tmp[7:9])
    hour = int(tmp[9:11])
    minute = int(tmp[11:13])
    sec = float(tmp[13:19])
    dt = datetime(year,month,day,hour,minute,int(sec//1),int(sec%1 * 1000000))
    
    lat = int(tmp[19:21])
    lat_m = float(tmp[21:26])
    lon = int(tmp[26:29])
    lon_m = float(tmp[29:34])
   
    lat+=lat_m/60
    lat=round(lat,2)
    lon+=lon_m/60
    lon=round(lon,2)
    
    depth = float(tmp[34:40])
    mag = float(tmp[40:44])
    n_sta = int(tmp[-4:])+1
    
    pfile_info = {}
    pfile_info["ori_time"] = str(dt)
    pfile_info["depth"] = depth
    pfile_info["mag"] = mag
    pfile_info["lat"] = lat
    pfile_info["lon"] = lon
    # ================================ header ================================= #
    # ================================= body ================================== #
    for i in range(1, n_sta):
        tmp = lines[i]
        sta = tmp[:7].strip()
        
        try:
            pdict = {}
            
            starttime = datetime(year=int(tmp[84:88]), month=int(tmp[88:90]), day=int(tmp[90:92]), hour=int(tmp[92:94]),
                                         minute=int(tmp[94:96]), second=int(tmp[96:98]), microsecond=int(tmp[99:101]))
            pdict['starttime'] = str(starttime)
            pdict['distance'] = float(tmp[8:14])
            pdict['Azimuth'] = int(tmp[14:17])
            pdict['p_polar'] = str(tmp[21])
            pdict['p_arrival_time'] = str(starttime + timedelta(seconds=float(tmp[23:30])))
            pdict['p_weight'] = int(tmp[31])
            pdict['s_arrival_time'] = str(starttime + timedelta(seconds=float(tmp[33:40])))
            pdict['s_weight'] = int(tmp[41])
            pdict['station_lat'] = float(tmp[42:50])
            pdict['station_lon'] = float(tmp[50:57])
            
            # instrument status
            if tmp[80] == '1':
                pdict['Z'] = False
            else:
                pdict['Z'] = True
                
            if tmp[81] == '1':
                pdict['N'] = False
            else:
                pdict['N'] = True
            
            if tmp[82] == '1':
                pdict['E'] = False
            else:
                pdict['E'] = True
            
            # 0: normal, 1: 可能有錯誤，但波型還是可以用, 2: 沒辦法判斷時間系統有沒有問題
            if tmp[83] == '!':
                pdict['time_system'] = 0
            elif tmp[83] == '*':
                pdict['time_system'] = 1
            elif tmp[83] == ' ':
                pdict['time_system'] = 2
                
            # Store wave file
            waveFile = sta + '_' + tmp[63:75] + '.txt'
            adict = getWavFile(os.path.join(root_path, waveFile), root_path)
            pdict['0'] = adict
            
            if pdict['p_polar'] == ' ':
                pdict['p_polar'] = 'unknown'    
        
        except Exception as e:
#             print(e)
            pass

        pfile_info[sta] = pdict
    
    return pfile_info

In [181]:
root_path = "/mnt/nas6/ori_TSMIP/"
p = unpackP(os.path.join(root_path, '24210827.20P'), root_path)

/mnt/nas6/ori_TSMIP/test


In [183]:
with open('/mnt/nas7/weiwei/test.json', 'w') as file:
    json.dump(p, file)